In [1]:
import pandas as pd
import os

In [2]:
# 設定 data_path
dir_data = './example/data-science-london-scikit-learn/'

# load Data
train_path = os.path.join(dir_data, 'train.csv')
train_label_path = os.path.join(dir_data, 'trainLabels.csv')

train_data = pd.read_csv(train_path, header=None)
train_label = pd.read_csv(train_label_path, header=None)
train_label.columns = ['Target']

test_path = os.path.join(dir_data, 'test.csv')
test_data = pd.read_csv(test_path, header=None)

print('train data shape: ', train_data.shape)
print('train label shape: ', train_label.shape)
print('test data shape: ', test_data.shape)

train data shape:  (1000, 40)
train label shape:  (1000, 1)
test data shape:  (9000, 40)


In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

# concat train data and target
df = pd.concat([train_data,train_label], axis=1)

# 進一步我們檢查變數在 Target 上的分布是否不同
plt.figure(figsize = (24, 24))

# 依不同 欄位 逐項繪製 KDE 圖形
for i in range(40):
    # 做 subplot
    plt.subplot(10, 4, i + 1)
    
    # KDE 圖形
    sns.kdeplot(df.loc[df['Target'] == 0, i], label = 'target == 0')
    sns.kdeplot(df.loc[df['Target'] == 1, i], label = 'target == 1')
    
    # 加上各式圖形標籤
    plt.title('Distribution of %s by Target Value' % i)
    plt.xlabel('%s' % i); plt.ylabel('Density');
    
plt.tight_layout(h_pad = 2.5)

/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1706: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [4]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import metrics

x_train, x_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.25, random_state=42)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Save Result 

In [5]:
y_pred_df = pd.DataFrame()

## GradientBoostingClassifier

In [6]:
clf = GradientBoostingClassifier()

# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 2, 3, 4, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

# 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

clf = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'], 
                                 n_estimators=grid_result.best_params_['n_estimators'])

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print('Accrucy:', metrics.accuracy_score(y_test, y_pred))

## append result
y_pred_df['GradientBoostingClassifier'] = clf.predict(test_data)


Fitting 3 folds for each of 15 candidates, totalling 45 fits


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best Accuracy: -0.125333 using {'max_depth': 4, 'n_estimators': 200}
Accrucy: 0.884


## AdaBoostClassifier

In [7]:
clf = AdaBoostClassifier()

# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
param_grid = dict(n_estimators=n_estimators)

# 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

clf = AdaBoostClassifier(n_estimators=grid_result.best_params_['n_estimators'])

clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

print('Accrucy:', metrics.accuracy_score(y_test, y_pred))

# append result
y_pred_df['AdaBoostClassifier'] = clf.predict(test_data)


Fitting 3 folds for each of 3 candidates, totalling 9 fits


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best Accuracy: -0.205333 using {'n_estimators': 300}


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accrucy: 0.744


In [8]:
from sklearn import svm

clf = svm.SVC()
print('svm parameter:\n\n', clf.fit(x_train, y_train))

y_pred = clf.predict(x_test)

print('\n Accrucy:', metrics.accuracy_score(y_test, y_pred))

# append result
y_pred_df['SVM'] = clf.predict(test_data)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


svm parameter:

 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

 Accrucy: 0.896


## Output Result

In [38]:
row_count = y_pred_df.shape[0]
columns = y_pred_df.columns.get_values()

result = []
for i in range(row_count):
    count = len([True for column in columns if y_pred_df[column][i] == 1])
    if(count >= 2):
        result.append(1)
    else:
        result.append(0)
        
result_df = pd.DataFrame(result, columns=["Solution"])
result_df.index += 1 
result_df.to_csv(os.path.join(dir_data, 'Result.csv'), index_label="Id")